# Machine learning application: Forecasting wind power. 

### Using alternative energy for social & enviromental Good

<table>
  <tr><td>
    <img src="https://github.com/dmatrix/mlflow-workshop-part-3/raw/master/images/wind_farm.jpg"
         alt="Keras NN Model as Logistic regression"  width="800">
  </td></tr>
</table>

In this notebook, we will use the MLflow Model Registry to build a machine learning application that forecasts the daily power output of a [wind farm](https://en.wikipedia.org/wiki/Wind_farm). 

Wind farm power output depends on weather conditions: generally, more energy is produced at higher wind speeds. Accordingly, the machine learning models used in the notebook predict power output based on weather forecasts with three features: `wind direction`, `wind speed`, and `air temperature`.

* This notebook uses altered data from the [National WIND Toolkit dataset](https://www.nrel.gov/grid/wind-toolkit.html) provided by NREL, which is publicly available and cited as follows:*

* Draxl, C., B.M. Hodge, A. Clifton, and J. McCaa. 2015. Overview and Meteorological Validation of the Wind Integration National Dataset Toolkit (Technical Report, NREL/TP-5000-61740). Golden, CO: National Renewable Energy Laboratory.*

* Draxl, C., B.M. Hodge, A. Clifton, and J. McCaa. 2015. "The Wind Integration National Dataset (WIND) Toolkit." Applied Energy 151: 355366.*

* Lieberman-Cribbin, W., C. Draxl, and A. Clifton. 2014. Guide to Using the WIND Toolkit Validation Code (Technical Report, NREL/TP-5000-62595). Golden, CO: National Renewable Energy Laboratory.*

* King, J., A. Clifton, and B.M. Hodge. 2014. Validation of Power Output for the WIND Toolkit (Technical Report, NREL/TP-5D00-61714). Golden, CO: National Renewable Energy Laboratory.*

<table>
  <tr><td>
    <img src="https://github.com/dmatrix/mlflow-workshop-part-3/raw/master/images/wind_farm.jpg"
         alt="Keras NN Model as Logistic regression"  width="800">
  </td></tr>
    <tr><td>
    <img src="https://github.com/dmatrix/mlflow-workshop-part-3/raw/master/images/wind_farm.jpg"
         alt="Keras NN Model as Logistic regression"  width="800">
  </td></tr>
</table>



In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import mlflow

### Run some class and utility notebooks 

This allows us to use some Python model classes and utility functions

In [3]:
%run ./rfr_class.ipynb
%run ./utils_class.ipynb

/Users/julesdamji/.conda/envs/tutorials/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Load our training data

In [4]:
# Load and print dataset
csv_path = "./data/windfarm_data.csv"

# Use column 0 (date) as the index
wind_farm_data = Utils.load_data(csv_path, index_col=0)
wind_farm_data.head(5)

temperature_00  wind_direction_00  wind_speed_00  temperature_08  \
2014-01-01        4.702022         106.742590       4.743292        7.189482   
2014-01-02        7.695733          98.036705       6.142715        9.977118   
2014-01-03        9.608235         274.061200      10.514304       10.840864   
2014-01-04        6.955563         257.910220       7.189170        5.317223   
2014-01-05        0.830547         265.394400       4.263086        2.480239   

            wind_direction_08  wind_speed_08  temperature_16  \
2014-01-01          100.41638       6.593832        8.172301   
2014-01-02           94.03181       4.383676        9.690135   
2014-01-03          242.87563      16.869741        8.991079   
2014-01-04          254.26170       9.069233        3.021174   
2014-01-05          104.79496       3.042063        4.227131   

            wind_direction_16  wind_speed_16      power  
2014-01-01           99.28800       5.967206  1959.3535  
2014-01-02          204.25444       1.696528  1266.6239  
2014-01-03          250.26830      12.038399  7545.6797  
2014-01-04          284.06537       4.590843  3791.0408  
2014-01-05          263.41690       3.899182   880.6115

### Get Validation data

In [5]:
X_train, y_train = Utils.get_training_data(wind_farm_data)
val_x, val_y = Utils.get_validation_data(wind_farm_data)

### Initialize a set of hyperparameters for the training and try three runs

In [6]:
# Initialize our model hyperparameters
params_list = [{"n_estimators": 100},
               {"n_estimators": 200},
               {"n_estimators": 300}]

In [7]:
# Train, fit and register our model and iterate over few different tuning parameters
# Use sqlite:///mlruns.db as the local store for tracking and model registery

mlflow.set_tracking_uri("sqlite:///mlruns.db")

model_name = "PowerForecastingModel"
for params in params_list:
  rfr = RFRModel.new_instance(params)
  print("Using paramerts={}".format(params))
  runID = rfr.mlflow_run(X_train, y_train, val_x, val_y, model_name, register=True)
  print("MLflow run_id={} completed with MSE={} and RMSE={}".format(runID, rfr.mse, rfr.rsme))

Using paramerts={'n_estimators': 100}


2020/09/24 17:56:01 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2020/09/24 17:56:01 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

MLflow run_id=3ef0f60a01cb424cb4e18920a97dbcc5 completed with MSE=45324.95891538796 and RMSE=212.8965920708642
Using paramerts={'n_estimators': 200}


INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Registered model 'PowerForecastingModel' already exists. Creating a new version of this model...
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Created

MLflow run_id=72a37671c91f4192a4d9be61a7950115 completed with MSE=45688.052429026866 and RMSE=213.74763724782284
Using paramerts={'n_estimators': 300}


INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Registered model 'PowerForecastingModel' already exists. Creating a new version of this model...
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Created

MLflow run_id=d56153f75ef14ea89a7508422b149af9 completed with MSE=43035.2370504347 and RMSE=207.44936020734


### Let's Examine the MLflow UI

1. From your local host where your started jupter start the mlflow ui
2. **mlflow ui --backend-store-uri sqlite:///mlruns.db**
3. Go to http://127.0.0.1:5000 in your browser
2. Let's examine some models and start comparing their metrics

# Integrating Model Registry with CI/CD Forecasting Application

<table>
  <tr><td>
    <img src="https://github.com/dmatrix/mlflow-workshop-part-3/raw/master/images/forecast_app.png"
         alt="Keras NN Model as Logistic regression"  width="800">
  </td></tr>
</table>

1. Use the model registry fetch different versions of the model
2. Score the model
3. Select the best scored model
4. Promote model to production, after testing

### Define a helper function to load PyFunc model from the registry
<table>
  <tr><td> Save a Keras Model Flavor and load as PyFunc Flavor</td></tr>
  <tr><td>
    <img src="https://raw.githubusercontent.com/dmatrix/mlflow-workshop-part-2/master/images/models_2.png"
         alt="" width="600">
  </td></tr>
</table>

In [8]:
def score_model(data, model_uri):
    model = mlflow.pyfunc.load_model(model_uri)
    return model.predict(data)

### Load scoring data

In [9]:
# Load the score data
score_path = "./data/score_windfarm_data.csv"
score_df = Utils.load_data(score_path, index_col=0)
score_df.head()

temperature_00  wind_direction_00  wind_speed_00  temperature_08  \
2020-12-27        7.123225         103.176630       8.133746        6.454002   
2020-12-28        5.376270         118.084330       5.558247        8.118839   
2020-12-29        8.593436         115.432590      12.181850        8.587968   
2020-12-30        8.069033         103.169685       9.983466        7.930485   

            wind_direction_08  wind_speed_08  temperature_16  \
2020-12-27         107.793220       6.326991        7.219884   
2020-12-28         116.193535       8.565966        9.307176   
2020-12-29         112.931360      11.970859        8.956771   
2020-12-30         106.045510       6.381556        8.228901   

            wind_direction_16  wind_speed_16      power  
2020-12-27         119.070526       3.062219  2621.4760  
2020-12-28         120.264430      11.993913  5423.6250  
2020-12-29         110.161095      11.301485  9132.1150  
2020-12-30         111.602160       4.087358  3667.9927

In [10]:
# Drop the power column since we are predicting that value
actual_power = pd.DataFrame(score_df.power.values, columns=['power'])
score = score_df.drop("power", axis=1)

In [11]:
# Formulate the model URI to fetch from the model registery
model_uri = "models:/{}/{}".format(model_name, 1)

# Predict the Power output 
pred_1 = pd.DataFrame(score_model(score, model_uri), columns=["predicted_1"])
pred_1

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


predicted_1
0  2891.701346
1  5125.912166
2  8690.109461
3  3812.213615

#### Combine with the actual power

In [12]:
actual_power["predicted_1"] = pred_1["predicted_1"]
actual_power

power  predicted_1
0  2621.4760  2891.701346
1  5423.6250  5125.912166
2  9132.1150  8690.109461
3  3667.9927  3812.213615

In [13]:
# Formulate the model URI to fetch from the model registery
model_uri = "models:/{}/{}".format(model_name, 2)

# Predict the Power output
pred_2 = pd.DataFrame(score_model(score, model_uri), columns=["predicted_2"])
pred_2

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


predicted_2
0  2768.066228
1  5150.133195
2  8745.217837
3  3826.041776

In [14]:
actual_power["predicted_2"] = pred_2["predicted_2"]
actual_power

power  predicted_1  predicted_2
0  2621.4760  2891.701346  2768.066228
1  5423.6250  5125.912166  5150.133195
2  9132.1150  8690.109461  8745.217837
3  3667.9927  3812.213615  3826.041776

In [15]:
# Formulate the model URI to fetch from the model registery
model_uri = "models:/{}/{}".format(model_name, 3)

# Formulate the model URI to fetch from the model registery
pred_3 = pd.DataFrame(score_model(score, model_uri), columns=["predicted_3"])
pred_3

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


predicted_3
0  2748.391785
1  5115.027138
2  8741.102441
3  3790.848239

### Combine the values into a single pandas DataFrame 

In [16]:
actual_power["predicted_3"] = pred_3["predicted_3"]
actual_power

power  predicted_1  predicted_2  predicted_3
0  2621.4760  2891.701346  2768.066228  2748.391785
1  5423.6250  5125.912166  5150.133195  5115.027138
2  9132.1150  8690.109461  8745.217837  8741.102441
3  3667.9927  3812.213615  3826.041776  3790.848239

### Let's Examine the MLflow UI

1. Go to http://127.0.0.1:5000 in your browser
2. Navigate to the Model Registry Page
3. Pick a version and transition stage: None->Production
